In [124]:
import scipy
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential, Model

from sklearn.metrics import mean_squared_error,mean_absolute_error


In [125]:
df=pd.read_csv('San Diego_featured_data.csv')
df.head()

,All sky irradiance,Clear sky irradiance,ALLSKY ICI,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX,CLRSKY_SFC_PAR_TOT
0,3.58,3.68,0.70,63.57,0.55,66.20
1,3.20,3.60,0.62,57.87,0.53,65.50
2,2.52,3.46,0.49,47.44,0.50,64.25
3,2.93,3.48,0.57,55.34,0.49,65.82
4,0.78,3.42,0.15,16.23,0.21,64.96


In [126]:
df.dropna(inplace=True)

In [127]:
y= df[['All sky irradiance']]
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(df)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten() 

In [128]:
X_train1, X_test1, Y_train, Ytest1 = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)


Train set shape (1533, 6)
testing set shape (384, 6)


In [129]:
X_train_series=np.reshape(X_train1,(1533,6,1))
X_valid_series=np.reshape(X_test1,(384,6,1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (1533, 6, 1)
Validation set shape (384, 6, 1)


In [130]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

In [131]:
model_lstm = Sequential()
model_lstm.add(LSTM(128, activation='relu', return_sequences=True,input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
#model_lstm.add(LSTM(units = 72,return_sequences=True,activation="relu"))    
#model_lstm.add(LSTM(units = 72,return_sequences=True,activation="relu"))     
model_lstm.add(LSTM(units = 128,return_sequences=True,activation="relu"))
model_lstm.add(LSTM(units = 128,activation="relu"))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(1,activation="linear"))
model_lstm.compile(loss='mae', optimizer=Adam(learning_rate=0.0001))
#model_lstm.summary()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [132]:
import time 
start= time.time()
lstm_history = model_lstm.fit(X_train_series, Y_train,epochs=100, batch_size=32,verbose=2)
end= time.time()
continential= end- start
print("convergence time:",continential) 

Epoch 1/100
48/48 - 2s - 35ms/step - loss: 0.8113
Epoch 2/100
48/48 - 0s - 9ms/step - loss: 0.7347
Epoch 3/100
48/48 - 1s - 11ms/step - loss: 0.3114
Epoch 4/100
48/48 - 1s - 11ms/step - loss: 0.1290
Epoch 5/100
48/48 - 1s - 11ms/step - loss: 0.1203
Epoch 6/100
48/48 - 1s - 11ms/step - loss: 0.1228
Epoch 7/100
48/48 - 1s - 11ms/step - loss: 0.1198
Epoch 8/100
48/48 - 1s - 11ms/step - loss: 0.1088
Epoch 9/100
48/48 - 1s - 11ms/step - loss: 0.1090
Epoch 10/100
48/48 - 1s - 12ms/step - loss: 0.1115
Epoch 11/100
48/48 - 1s - 11ms/step - loss: 0.1067
Epoch 12/100
48/48 - 1s - 11ms/step - loss: 0.1096
Epoch 13/100
48/48 - 1s - 12ms/step - loss: 0.1038
Epoch 14/100
48/48 - 1s - 11ms/step - loss: 0.1013
Epoch 15/100
48/48 - 1s - 11ms/step - loss: 0.1065
Epoch 16/100
48/48 - 1s - 11ms/step - loss: 0.1043
Epoch 17/100
48/48 - 1s - 11ms/step - loss: 0.1027
Epoch 18/100
48/48 - 1s - 11ms/step - loss: 0.0994
Epoch 19/100
48/48 - 1s - 11ms/step - loss: 0.0990
Epoch 20/100
48/48 - 1s - 11ms/step - los

In [133]:
import time
start= time.time()
prediction_lstm=model_lstm.predict(X_valid_series)
end= time.time()
continential= end -start
print("inference time:",continential)
prediction_lstm

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  
inference time: 0.2917008399963379


array([[-1.4362061 ],
       [-0.8216366 ],
       [-0.03704483],
       [-0.31030682],
       [ 0.87776977],
       [-1.2454668 ],
       [ 0.47108924],
       [-0.4543164 ],
       [ 0.85015965],
       [ 0.18220754],
       [ 0.8678809 ],
       [ 0.9050526 ],
       [ 0.02701021],
       [ 0.96472925],
       [ 1.1517061 ],
       [ 0.84346455],
       [ 0.8933967 ],
       [ 1.2164634 ],
       [-0.7265576 ],
       [-0.24782443],
       [ 0.4389837 ],
       [ 1.0220668 ],
       [-0.20430015],
       [ 0.6367458 ],
       [-0.7284281 ],
       [-0.82450384],
       [ 0.74819535],
       [-0.4954366 ],
       [-0.74753696],
       [ 0.5674026 ],
       [-0.8255417 ],
       [ 0.23160614],
       [ 1.3824264 ],
       [ 1.7046115 ],
       [ 1.6175587 ],
       [ 0.42805564],
       [ 0.8684321 ],
       [-0.13990763],
       [ 0.11497556],
       [ 1.1672225 ],
       [ 1.6207416 ],
       [ 1.5900586 ],
       [ 1.4849079 ],
       [ 1.288342  ],
       [ 0.98406154],
       [ 1

In [134]:
prediction_lstm.shape

(384, 1)

In [135]:
predictions = scaler_y.inverse_transform(prediction_lstm)
Actual = scaler_y.inverse_transform(Ytest1.reshape(-1, 1))

In [136]:
Calculated =pd.DataFrame(predictions, columns = ['Prediction'])
Actual=pd.DataFrame(Actual, columns=["Actual"])

In [137]:
import os
import pandas as pd

# Concatenate Calculated and Actual DataFrames along the columns
combined = pd.concat([Calculated, Actual], axis=1)

# Define the path to the existing folder
save_dir = "/Users/ubaidahmed/Desktop/Daily_SI_forecasting /San Diego/Models/LSTM"

# Define the filename within the specified folder
filename = os.path.join(save_dir, "LSTM_output.csv")

# Save the combined DataFrame to CSV
combined.to_csv(filename, index=False)

print(f"File saved to {filename}")


File saved to /Users/ubaidahmed/Desktop/Daily_SI_forecasting /San Diego/Models/LSTM/LSTM_output.csv


# Error Calculation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("LSTM_output.csv")
features= list(df)
features

['Prediction', 'Actual']

In [3]:
df=df*1000

In [4]:
z= df["Actual"]
x= df["Prediction"]

In [5]:
import math
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(z,x)
RMSE= math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE= mean_absolute_error(z,x)
from sklearn.metrics import mean_absolute_percentage_error
MAPE= (mean_absolute_percentage_error(z,x))*100
print("Mean Square Error is:",MSE)
print("Root Mean Square Error is:",RMSE)
print("Mean Absolute Error is:",MAE)
print("Mean Absolute Percentage Error is:",MAPE)

Mean Square Error is: 5682.15501558833
Root Mean Square Error is: 75.3800704137926
Mean Absolute Error is: 47.205848437499974
Mean Absolute Percentage Error is: 0.9953158027040647


In [6]:
max=x.max()
min=x.min()
mean=x.mean()

print("Max",max)
print("Min",min)
print('Mean',mean)

Max 9059.127
Min 1184.7528
Mean 5215.863817708333


In [7]:
NRMSE= (RMSE/(max-min))*100
print('Normalized Root Mean Square Error is:',NRMSE)

Normalized Root Mean Square Error is: 0.9572833154638827


In [8]:
NRMSE= (RMSE/(mean))*100
print('Normalized Root Mean Square Error is:',NRMSE)

Normalized Root Mean Square Error is: 1.4452077939203547


In [9]:
relative_errors = (z - x) / z
squared_relative_errors = relative_errors ** 2
abs_relative_errors = np.abs(relative_errors)

In [10]:
msre = np.mean(squared_relative_errors)
rmsre = np.sqrt(msre)
mare = np.mean(abs_relative_errors)
rmspe = np.sqrt(np.mean((abs_relative_errors * 100) ** 2))

In [11]:
print("Mean Square Relative Error is:",msre)
print("Root Mean Square Relative Error is:",rmsre)
print("Mean Absolute Relative Error is:",mare)
print("Root Mean Square Percentage Error is:",rmspe)

Mean Square Relative Error is: 0.00024242398462983022
Root Mean Square Relative Error is: 0.015569970604655302
Mean Absolute Relative Error is: 0.009953158027040648
Root Mean Square Percentage Error is: 1.5569970604655303
